In [1]:
import os
from binascii import hexlify
from time import sleep
import datetime

In [2]:
import tagnet

TagNet Driver Version 0.2.1


In [3]:
from tagdump import tagdump as td

### influxdb host: localhost, port: 8086, user: root, password: root, dbname: test
### influxdb failed to respond to ping


In [4]:
tempfile     = 'tempdata'
tagdir       = '/home/pi/tags/1fbcd99fd29f'
dblkdir      = tagdir+'/tag/sd/0/dblk' 
dblkbyte     = dblkdir+'/byte' 
dblkresync   = dblkdir+'/.resync'
dblklastsync = dblkdir+'/.last_sync'
dblkrecnum   = dblkdir+'/.recnum'

In [5]:
!pwd

/mnt/neptune/TagNet/notebooks/TagFuse_Tutorial


In [6]:
def int32(x):
  if x>0xFFFFFFFF:
    raise OverflowError
  if x>0x7FFFFFFF:
    x=int(0x100000000-x)
    if x<2147483648:
      return -x
    else:
      return -2147483648
  return x

In [7]:
def round_to_word(a):
    return ((a-(a%4)) if (a%4) else (a))

In [8]:
def get_file_size(fn):
    ret=!stat  -c '%s' $fn
    offset=int(ret[0])
    int32(offset)
    return offset

In [9]:
def err_by_value(val):
    for e in tagnet.tlv_errors:
        if e.value == val: break
    return e.name

In [10]:
def size_to_err(pos):
    ipos = int32(pos)
    if ipos < 0:
        print('error', -ipos, err_by_value(-ipos))
        return -ipos
    return ipos

# Search for Sync records from file

In [11]:
!dd count=100 bs=1k if=$dblkbyte of=tempdata status=progress

92160 bytes (92 kB, 90 KiB) copied, 1.00933 s, 91.3 kB/s
100+0 records in
100+0 records out
102400 bytes (102 kB, 100 KiB) copied, 1.27343 s, 80.4 kB/s


In [12]:
infile  = td.TagFile(open(tempfile))

In [13]:
td.resync(infile, 0)

512

In [14]:
offset = 0
for i in range(100):
    try:
        offset = td.resync(infile, offset)
    except:
        break
    print(i, offset)
    offset += 4

(0, 512)
(1, 4328)
(2, 6596)
(3, 8864)
(4, 11092)
(5, 13400)
(6, 15628)
(7, 17816)
(8, 20044)
(9, 22232)
(10, 24460)
(11, 26648)
(12, 28876)
(13, 31104)
(14, 33292)
(15, 35520)
(16, 37708)
(17, 39936)
(18, 44036)
(19, 46304)
(20, 46332)
(21, 46360)
(22, 46908)
(23, 47456)
(24, 47484)
(25, 47512)
(26, 47540)
(27, 47568)
(28, 47596)
(29, 48704)
(30, 48732)
(31, 48760)
(32, 48788)
(33, 48816)
(34, 48844)
(35, 51512)
(36, 51540)
(37, 51568)
(38, 51596)
(39, 51624)
(40, 51652)
(41, 54120)
(42, 54148)
(43, 54176)
(44, 54204)
(45, 54232)
(46, 54260)
(47, 54568)
(48, 54596)
(49, 54624)
(50, 54652)
(51, 54680)
(52, 54708)
(53, 55016)
(54, 55044)
(55, 55072)
(56, 55100)
(57, 55368)
(58, 55396)
(59, 55424)
(60, 55452)
(61, 55480)
(62, 55508)
(63, 55776)
(64, 55804)
(65, 55832)
(66, 56316)
(67, 59936)
(68, 64012)
(69, 67912)
(70, 71748)
(71, 75808)
(72, 79348)
(73, 82952)
(74, 83740)
(75, 83768)
(76, 83796)
(77, 83824)
(78, 83852)
(79, 83920)
(80, 86148)
(81, 86176)
(82, 86204)
(83, 88032)
(84, 88

# Search for Sync records from remote tag

In [15]:
infile  = td.TagFile(open(dblkbyte), net_io = True)

In [16]:
td.resync(infile, 0)

512L

In [17]:
td.get_record(infile)

(512L,
 len: 28  type: 7  hdr_crc8: 204  recnum: 1  [rt: sub_sec: 8272  sec: 12  min: 5  hr: 22  dow: 4  day: 9  mon: 11  year: 2018]
   recsum: 0x0534,
 bytearray(b'\x1c\x00\x07\xcc\x01\x00\x00\x00P \x0c\x05\x16\x04\t\x0b\xe2\x074\x05\x00\x00\x00\x00\xef\x00\xdf\xde'))

In [18]:
offset = 0
for i in range(10):
    offset += 4
    offset = size_to_err(td.resync(infile, offset))
    print(i, td.get_record(infile))

(0, (512L, len: 28  type: 7  hdr_crc8: 204  recnum: 1  [rt: sub_sec: 8272  sec: 12  min: 5  hr: 22  dow: 4  day: 9  mon: 11  year: 2018]
  recsum: 0x0534, bytearray(b'\x1c\x00\x07\xcc\x01\x00\x00\x00P \x0c\x05\x16\x04\t\x0b\xe2\x074\x05\x00\x00\x00\x00\xef\x00\xdf\xde')))
(1, (4328L, len: 28  type: 3  hdr_crc8: 230  recnum: 84  [rt: sub_sec: 12661  sec: 14  min: 10  hr: 22  dow: 4  day: 9  mon: 11  year: 2018]
  recsum: 0x05dc, bytearray(b'\x1c\x00\x03\xe6T\x00\x00\x00u1\x0e\n\x16\x04\t\x0b\xe2\x07\xdc\x05\x00\x02\x00\x00\xef\x00\xdf\xde')))
(2, (6596L, len: 28  type: 3  hdr_crc8: 216  recnum: 141  [rt: sub_sec: 12661  sec: 14  min: 15  hr: 22  dow: 4  day: 9  mon: 11  year: 2018]
  recsum: 0x0702, bytearray(b'\x1c\x00\x03\xd8\x8d\x00\x00\x00u1\x0e\x0f\x16\x04\t\x0b\xe2\x07\x02\x07\xe8\x10\x00\x00\xef\x00\xdf\xde')))
(3, (8864L, len: 28  type: 3  hdr_crc8: 125  recnum: 198  [rt: sub_sec: 12661  sec: 14  min: 20  hr: 22  dow: 4  day: 9  mon: 11  year: 2018]
  recsum: 0x06ca, bytearray(b

In [19]:
get_file_size(dblkresync)

22232

In [20]:
get_file_size(dblklastsync)

944320

# Compare Syncs between file and network

In [21]:
diskfile = td.TagFile(open(tempfile))
netfile  = td.TagFile(open(dblkbyte), net_io = True)

In [24]:
get_file_size(tempfile)

102400

In [22]:
offset = get_file_size(tempfile) / 2
for i in range(10):
    disk_off = td.resync(diskfile, offset)
    net_off = td.resync(netfile, offset)
    print(i, offset, disk_off, net_off)
    if net_off != disk_off:
        print('*** ERROR disk: (), net: {}'.format(disk_off, net_off))
    offset = disk_off + 4

(0, 51200, 51512, 51512L)
(1, 51516, 51540, 51540L)
(2, 51544, 51568, 51568L)
(3, 51572, 51596, 51596L)
(4, 51600, 51624, 51624L)
(5, 51628, 51652, 51652L)
(6, 51656, 54120, 54120L)
(7, 54124, 54148, 54148L)
(8, 54152, 54176, 54176L)
(9, 54180, 54204, 54204L)


In [25]:
diskfile.seek(0,2)
diskfile.tell()

102400L

In [28]:
netfile.seek(0,2)
netfile.tell()

945668L